In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, denoise
from cellpose.io import imread
from glob import glob

In [ ]:
# list of files
# PUT PATH TO YOUR FILES HERE!
files = list(sorted(glob('data/growth-2/*.png')))

imgs = [imread(f) for f in files]

In [ ]:
io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(gpu = True, model_type='cyto3')
dn = denoise.DenoiseModel(gpu=True, model_type="denoise_cyto3")

diameter = 20
nimg = len(imgs)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images

imgs = dn.eval(imgs, channels=channels, diameter=diameter)
masks, flows, styles, diams = model.eval(imgs, diameter=diameter, channels=channels)

In [ ]:
import matplotlib
import cv2 as cv
import multiprocessing as mp
import tqdm
import os
from pathlib import Path
from PIL import Image

def __get_marked_folder_name(img_filename, new_suffix):
    folder_name = os.path.dirname(img_filename)
    folder_name = os.path.dirname(img_filename)
    if folder_name[-1] == "/":
        folder_name = folder_name[:-1]
    save_file_name = str(os.path.basename(img_filename).split(".png")[0]) + str(new_suffix)
    save_path = Path("{}-marked".format(folder_name))
    return save_path, save_file_name

def plot_iteration(img_filename, img, markers):
    fig, ax = plt.subplots(figsize=(12, 8))

    ax.imshow(img)
    for n_marker in np.unique(markers)[1:]:
        x, y = np.nonzero((markers==n_marker).T)
        coords = np.array([x, y])
        middle = np.mean(coords, axis=1)
        circle = matplotlib.patches.Circle(
            middle,
            radius=10,
            facecolor="white",
            edgecolor="k",
        )
        ax.add_patch(circle)
        label = ax.annotate(
            str(int(n_marker)),
            xy=middle,
            fontsize=10,
            verticalalignment="center",
            horizontalalignment="center"
        )
    ax.contour(markers, len(np.unique(markers)))
    save_path, save_file_name = __get_marked_folder_name(img_filename, "-marked.png")
    os.makedirs(save_path, exist_ok=True)
    save_markers(img_filename, markers)
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig("{}/{}".format(save_path, save_file_name), bbox_inches='tight')

def __plot_iteration_helper(filename_img_markers):
    filename, img, markers = filename_img_markers
    return plot_iteration(filename, img, markers)

def save_markers(img_filename, markers):
    save_path, save_filename = __get_marked_folder_name(img_filename, "-markers.csv")
    np.savetxt(Path(save_path) / save_filename, markers, delimiter=",")
    im = Image.fromarray(markers, mode="L")
    save_path, save_filename2 = __get_marked_folder_name(img_filename, "-markers.tif")
    im.save(Path(save_path) / save_filename2)

def load_masks(save_path):
    markers = []
    for filename in sorted(glob(str(save_path) + "/*.csv")):
        markers.append(np.loadtxt(filename, delimiter=","))
    return markers

masks = load_masks("data/growth-2-marked")

In [ ]:
import scipy as sp

# TODO we need to specify these units
minutes_per_iteration = 1

y = np.array([len(np.unique(m)) for m in masks])
x = np.arange(len(y)) * minutes_per_iteration

# Clean up data: Exclude datapoints where next iteration is smaller than previous
current_max = [y[0]]
def filtering(i, current_max):
    if y[i] >= current_max[0]:
        current_max[0] = [y[i]]
        return True
    else:
        return False

filt = np.array([filtering(i, current_max) for i in range(len(y))])
x_nop = x[filt==False]
y_nop = y[filt==False]
x = x[filt]
y = y[filt]

def exp_growth(t, t0, l):
    s = np.maximum(t-t0, 0)
    return y[0] * np.exp(l*s)

popt, _ = sp.optimize.curve_fit(exp_growth, x, y, p0=(50 * minutes_per_iteration, 0.15 / minutes_per_iteration))

fig, ax = plt.subplots()
ax.plot(x, y, color="#50AA50")
ax.scatter(x_nop, y_nop, color="orange", marker="x")
ax.plot(
    x,
    exp_growth(x, *popt),
    label='Delayed Exponential Growth:\n' +
        '$N(t)=N_0 \\text{exp}(\lambda t\Theta(t-t_0))$\n' +
        "with $\lambda={:6.4f}".format(popt[1]) + '/\\text{min}$' +
        " and $t_0={:2.0f}".format(popt[0]) + '\\text{min}$',
    linestyle="--",
    color="gray",
)
ax.set_xlabel("Time [min]")
ax.set_ylabel("Number of cells")
ax.legend()
plt.show()

In [ ]:
# Only plot values which have been labelled correctly
pool = mp.Pool()
files_filtered = np.array(files)# np.array(files)[filt]
imgs_filtered = np.array(imgs)# np.array(imgs)[filt]
masks_filtered = np.array(masks)[filt]

_ = list(tqdm.tqdm(pool.imap(
        __plot_iteration_helper,
        zip(files_filtered, imgs_filtered, [m for m in masks_filtered])
    ),
    total=len(files_filtered))
)

In [ ]:
def create_lineage(masks) -> dict:
    lineages = {}
    for n, mask in enumerate(masks[0]):
        print(n, np.sum(mask))
    pass

create_lineage(masks[:50])

In [ ]:
# Make movie
import os
bashcmd = "ffmpeg\
    -pattern_type glob\
    -i 'data/growth-2-marked/*-marked.png'\
    -y\
    -c:v h264\
    -r 15\
    -pix_fmt yuv420p\
    growth-2-markers.mp4"
os.system(bashcmd)

## Further Steps
1. [ ] Make segmentation stable in time
    1. [ ] Assign identifiers to cells which are persistent in time
    2. [ ] Assign completely new identifiers to cells resulting from division (but link them to their parent cell)
2. [ ] Construct a function which takes a number of cells with position, shape, etc. and returns a  
    prediction depending on their parameters.
    ```python
    def predict_cells(cells: list[Cell]) -> list[Cell]:
        pass
    ```
3. [ ] Compare predictions and actual values and try to determine all the parameters in the 
    `predict_cells` function.